In [11]:
import pandas as pd
import numpy as np
from glob import glob
import os

In [12]:

file_lst = glob("raw/*.csv")


In [13]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [14]:

n_normal = 0
n_attack = 0
req_ft=['duration','src_bytes','dst_bytes','missed_bytes','src_pkts','dst_pkts','label']
print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, low_memory=False)  
    df = df[df['type'].isin(['normal', 'dos', 'ddos'])]
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['label'] == 1]
    nr_df = df[df['label'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]    

    print(len(r))    
    
    rng = np.random.default_rng()
    index=np.sort(rng.choice([i for i in range(size)], size=int(size*0.1), replace=False, shuffle=False))
    r.add(df.to_numpy()[index])
    del df
    


7
0
100000
200000
300000
301440
388920
488920
491028
491393
512260
579513
679513
740686
741257
741539
742288
842288
865786
927073
1027073
1027698
1028190


In [15]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

NORMAL: 755124 ATTACK: 9540336


In [16]:
df = r.getDf()
if not os.path.exists('processed'):
        os.makedirs('processed')    
    
df.to_csv("processed/train_data.csv", index=False)

In [17]:
df.tail(15)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,dst_pkts,label
1029522,0.000575,31.0,106.0,0.0,1.0,1.0,0.0
1029523,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
1029524,0.000528,31.0,106.0,0.0,1.0,1.0,0.0
1029525,0.000700,31.0,106.0,0.0,1.0,1.0,0.0
1029526,0.000517,31.0,106.0,0.0,1.0,1.0,0.0
1029527,0.000553,0.0,212.0,0.0,0.0,2.0,0.0
1029528,0.000373,31.0,106.0,0.0,1.0,1.0,0.0
1029529,0.000438,0.0,212.0,0.0,0.0,2.0,0.0
1029530,0.000507,31.0,106.0,0.0,1.0,1.0,0.0
1029531,0.000514,31.0,106.0,0.0,1.0,1.0,0.0
